In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
# model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-02-13-srev-curri/OML-order1-id4-2023-02-13_05-46-13.147873.pt"
model_path = "/data/model_runs/original_oml/aOML-order4-inlr005-2023-02-16-srev-curri/OML-order4-id4-2023-02-17_00-51-13.202492.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497510_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order4-inlr005-2023-02-16-srev-curri/OML-order4-id4-2023-02-17_00-51-13.202535_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*2,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-02-17 17:06:29,947 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2023-02-17 17:06:31,194 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2023-02-17 17:06:31,197 - transformers.configuration_utils - INFO - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [ ]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.5562, accuracy = 0.7688, precision = 0.7740, recall = 0.7567, F1 score = 0.7571
Query set metrics: Loss = 0.8276, accuracy = 0.6250, precision = 0.6267, recall = 0.6267, F1 score = 0.6133
Query ID 1/475
Support set metrics: Loss = 0.5186, accuracy = 0.8063, precision = 0.8155, recall = 0.7967, F1 score = 0.7981
Query set metrics: Loss = 0.8165, accuracy = 0.7500, precision = 0.7000, recall = 0.8000, F1 score = 0.7085
Query ID 2/475
Support set metrics: Loss = 0.5891, accuracy = 0.7562, precision = 0.7732, recall = 0.7450, F1 score = 0.7416
Query set metrics: Loss = 1.1972, accuracy = 0.5000, precision = 0.4833, recall = 0.5167, F1 score = 0.4848
Query ID 3/475
Support set metrics: Loss = 0.4927, accuracy = 0.8063, precision = 0.8092, recall = 0.8000, F1 score = 0.7985
Query set metrics: Loss = 1.4791, accuracy = 0.6875, precision = 0.7667, recall = 0.7000, F1 score = 

Support set metrics: Loss = 0.4789, accuracy = 0.8438, precision = 0.8503, recall = 0.8383, F1 score = 0.8400
Query set metrics: Loss = 0.6985, accuracy = 0.8125, precision = 0.8042, recall = 0.8250, F1 score = 0.8071
Query ID 36/475
Support set metrics: Loss = 0.4294, accuracy = 0.8187, precision = 0.8292, recall = 0.8067, F1 score = 0.8083
Query set metrics: Loss = 0.5914, accuracy = 0.8125, precision = 0.7667, recall = 0.7667, F1 score = 0.7133
Query ID 37/475
Support set metrics: Loss = 0.4989, accuracy = 0.8125, precision = 0.8218, recall = 0.8000, F1 score = 0.8004
Query set metrics: Loss = 0.9951, accuracy = 0.5625, precision = 0.5600, recall = 0.6100, F1 score = 0.5438
Query ID 38/475
Support set metrics: Loss = 0.4813, accuracy = 0.7875, precision = 0.8038, recall = 0.7750, F1 score = 0.7760
Query set metrics: Loss = 0.9846, accuracy = 0.6250, precision = 0.5100, recall = 0.5867, F1 score = 0.5378
Query ID 39/475
Support set metrics: Loss = 0.4949, accuracy = 0.8250, precision

Query set metrics: Loss = 1.3846, accuracy = 0.3750, precision = 0.3133, recall = 0.3600, F1 score = 0.3289
Query ID 71/475
Support set metrics: Loss = 0.5230, accuracy = 0.7937, precision = 0.8071, recall = 0.7800, F1 score = 0.7812
Query set metrics: Loss = 0.9062, accuracy = 0.7500, precision = 0.6300, recall = 0.6500, F1 score = 0.6111
Query ID 72/475
Support set metrics: Loss = 0.5923, accuracy = 0.7250, precision = 0.7322, recall = 0.7133, F1 score = 0.7107
Query set metrics: Loss = 0.9738, accuracy = 0.6250, precision = 0.6833, recall = 0.7400, F1 score = 0.6900
Query ID 73/475
Support set metrics: Loss = 0.4937, accuracy = 0.7937, precision = 0.7993, recall = 0.7833, F1 score = 0.7851
Query set metrics: Loss = 1.5598, accuracy = 0.4375, precision = 0.4200, recall = 0.4233, F1 score = 0.3881
Query ID 74/475
Support set metrics: Loss = 0.4955, accuracy = 0.7875, precision = 0.7985, recall = 0.7800, F1 score = 0.7813
Query set metrics: Loss = 1.3028, accuracy = 0.4375, precision =

Query set metrics: Loss = 0.9289, accuracy = 0.5000, precision = 0.5000, recall = 0.5000, F1 score = 0.4667
Query ID 106/475
Support set metrics: Loss = 0.5683, accuracy = 0.7750, precision = 0.7861, recall = 0.7633, F1 score = 0.7636
Query set metrics: Loss = 1.4678, accuracy = 0.4375, precision = 0.3600, recall = 0.5333, F1 score = 0.4133
Query ID 107/475
Support set metrics: Loss = 0.5041, accuracy = 0.7875, precision = 0.7978, recall = 0.7750, F1 score = 0.7761
Query set metrics: Loss = 1.2133, accuracy = 0.6250, precision = 0.6833, recall = 0.6167, F1 score = 0.6057
Query ID 108/475
Support set metrics: Loss = 0.5188, accuracy = 0.7937, precision = 0.8030, recall = 0.7800, F1 score = 0.7827
Query set metrics: Loss = 1.4629, accuracy = 0.5000, precision = 0.5167, recall = 0.5700, F1 score = 0.5000
Query ID 109/475
Support set metrics: Loss = 0.5500, accuracy = 0.7812, precision = 0.7938, recall = 0.7750, F1 score = 0.7757
Query set metrics: Loss = 1.4676, accuracy = 0.5000, precisi

Query set metrics: Loss = 0.6365, accuracy = 0.7500, precision = 0.8167, recall = 0.7267, F1 score = 0.7435
Query ID 141/475
Support set metrics: Loss = 0.5472, accuracy = 0.7750, precision = 0.8122, recall = 0.7667, F1 score = 0.7701
Query set metrics: Loss = 0.9897, accuracy = 0.6250, precision = 0.5067, recall = 0.5500, F1 score = 0.5181
Query ID 142/475
Support set metrics: Loss = 0.5260, accuracy = 0.7688, precision = 0.7789, recall = 0.7583, F1 score = 0.7565
Query set metrics: Loss = 0.8806, accuracy = 0.8125, precision = 0.6833, recall = 0.7500, F1 score = 0.7100
Query ID 143/475
Support set metrics: Loss = 0.5075, accuracy = 0.7688, precision = 0.7718, recall = 0.7533, F1 score = 0.7534
Query set metrics: Loss = 0.6850, accuracy = 0.7500, precision = 0.8133, recall = 0.8100, F1 score = 0.7321
Query ID 144/475
Support set metrics: Loss = 0.4831, accuracy = 0.8250, precision = 0.8345, recall = 0.8167, F1 score = 0.8175
Query set metrics: Loss = 1.4380, accuracy = 0.4375, precisi

...

Support set metrics: Loss = 0.0896, accuracy = 0.9750, precision = 0.9807, recall = 0.9821, F1 score = 0.9805
Query set metrics: Loss = 0.0049, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 70/475
Support set metrics: Loss = 0.0881, accuracy = 0.9688, precision = 0.9766, recall = 0.9714, F1 score = 0.9729
Query set metrics: Loss = 0.1865, accuracy = 0.9375, precision = 0.8889, recall = 0.8889, F1 score = 0.8889
Query ID 71/475
Support set metrics: Loss = 0.0580, accuracy = 0.9812, precision = 0.9836, recall = 0.9821, F1 score = 0.9821
Query set metrics: Loss = 0.0129, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 72/475
Support set metrics: Loss = 0.1018, accuracy = 0.9500, precision = 0.9598, recall = 0.9643, F1 score = 0.9610
Query set metrics: Loss = 0.0039, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 73/475
Support set metrics: Loss = 0.0754, accuracy = 0.9750, precision

Support set metrics: Loss = 0.0801, accuracy = 0.9625, precision = 0.9688, recall = 0.9714, F1 score = 0.9696
Query set metrics: Loss = 0.0041, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 105/475
Support set metrics: Loss = 0.0858, accuracy = 0.9750, precision = 0.9802, recall = 0.9786, F1 score = 0.9786
Query set metrics: Loss = 0.0804, accuracy = 0.9375, precision = 0.8636, recall = 0.9091, F1 score = 0.8788
Query ID 106/475
Support set metrics: Loss = 0.1010, accuracy = 0.9750, precision = 0.9830, recall = 0.9786, F1 score = 0.9798
Query set metrics: Loss = 0.2318, accuracy = 0.9375, precision = 0.9500, recall = 0.9800, F1 score = 0.9556
Query ID 107/475
Support set metrics: Loss = 0.0667, accuracy = 0.9875, precision = 0.9899, recall = 0.9929, F1 score = 0.9912
Query set metrics: Loss = 0.0233, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 108/475
Support set metrics: Loss = 0.0886, accuracy = 0.9625, preci

Support set metrics: Loss = 0.0861, accuracy = 0.9750, precision = 0.9761, recall = 0.9786, F1 score = 0.9768
Query set metrics: Loss = 0.0068, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 140/475
Support set metrics: Loss = 0.1062, accuracy = 0.9688, precision = 0.9765, recall = 0.9714, F1 score = 0.9726
Query set metrics: Loss = 0.2212, accuracy = 0.9375, precision = 1.0000, recall = 0.9545, F1 score = 0.9697
Query ID 141/475
Support set metrics: Loss = 0.1314, accuracy = 0.9625, precision = 0.9739, recall = 0.9643, F1 score = 0.9667
Query set metrics: Loss = 0.0055, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 142/475
Support set metrics: Loss = 0.0943, accuracy = 0.9625, precision = 0.9751, recall = 0.9750, F1 score = 0.9738
Query set metrics: Loss = 0.0046, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 143/475
Support set metrics: Loss = 0.1238, accuracy = 0.9437, preci

Support set metrics: Loss = 0.1561, accuracy = 0.9437, precision = 0.9588, recall = 0.9536, F1 score = 0.9522
Query set metrics: Loss = 0.0044, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 175/475
Support set metrics: Loss = 0.1000, accuracy = 0.9688, precision = 0.9768, recall = 0.9750, F1 score = 0.9747
Query set metrics: Loss = 0.0046, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 176/475
Support set metrics: Loss = 0.0404, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query set metrics: Loss = 0.3010, accuracy = 0.9375, precision = 0.8889, recall = 0.8889, F1 score = 0.8889
Query ID 177/475
Support set metrics: Loss = 0.0766, accuracy = 0.9812, precision = 0.9862, recall = 0.9893, F1 score = 0.9875
Query set metrics: Loss = 0.0058, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 178/475
Support set metrics: Loss = 0.0465, accuracy = 0.9875, preci

Support set metrics: Loss = 0.1388, accuracy = 0.9563, precision = 0.9680, recall = 0.9571, F1 score = 0.9591
Query set metrics: Loss = 0.0060, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 210/475
Support set metrics: Loss = 0.1037, accuracy = 0.9563, precision = 0.9679, recall = 0.9750, F1 score = 0.9679
Query set metrics: Loss = 0.0098, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 211/475
Support set metrics: Loss = 0.1104, accuracy = 0.9625, precision = 0.9669, recall = 0.9643, F1 score = 0.9643
Query set metrics: Loss = 0.0260, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 212/475
Support set metrics: Loss = 0.1018, accuracy = 0.9563, precision = 0.9647, recall = 0.9643, F1 score = 0.9624
Query set metrics: Loss = 0.0042, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 213/475
Support set metrics: Loss = 0.1365, accuracy = 0.9313, preci

In [ ]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

In [ ]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")